In [ ]:
import ollama

In [ ]:
def list_available_models():
    """Lists the available models in Ollama along with their file sizes."""
    try:
        models_info = ollama.list()['models']
        if not models_info:
            print("No models found in Ollama.")
            return []

        print("\n--- Available Ollama Models ---")
        model_names = []
        for model in models_info:
            name = model['model']
            file_size = model['size']
            family = model['details']['family']
            param_size = model['details']['parameter_size']
            quant_level = model['details']['quantization_level']
            human_readable_size = _format_bytes(file_size)
            print(f"Model: {name}, File size: {human_readable_size}, Family: {family}, Parameter size: {param_size}, Parameter size: {quant_level}")
            model_names.append(name)
        return model_names
    except Exception as e:
        print(f"Error listing models: {e}")
        return []

In [ ]:
def _format_bytes(size_bytes):
    """Formats bytes into a human-readable string (KB, MB, GB)."""
    if size_bytes == 0:
        return "0 bytes"
    i = 0
    power = 1024
    size_labels = ["bytes", "KB", "MB", "GB", "TB"]
    while size_bytes >= power and i < len(size_labels) - 1:
        size_bytes /= power
        i += 1
    return f"{size_bytes:.2f} {size_labels[i]}"

In [ ]:
available_models = list_available_models()

In [ ]:
def run_ollama_query(model_name, prompt):
    """Sends a prompt to the specified Ollama model and returns the response."""
    try:
        response = ollama.chat(
            model=model_name,
            messages=[{'role': 'user', 'content': prompt}]
        )
        return response['message']['content'].strip()
    except Exception as e:
        print(f"Error querying model '{model_name}': {e}")
        return None

In [ ]:
base_model = "gemma3:1b"  # Ensure this is available
pirate_model_name = "mypirate"
low_temp_model_name = "mypiratelowtemp"
high_temp_model_name = "mypiratehightemp"
modelfile_content = f"""
FROM {base_model}
SYSTEM \"\"\"You are a helpful assistant that always responds in the style of a pirate.\"\"\"
"""

# Create Modelfiles
with open("Modelfile_base", "w") as f:
    f.write(modelfile_content + "PARAMETER temperature 0.8")
with open("Modelfile_low_temp", "w") as f:
    f.write(modelfile_content + "PARAMETER temperature 0.2")
with open("Modelfile_high_temp", "w") as f:
    f.write(modelfile_content + "PARAMETER temperature 1.2")

# Create the custom models (you only need to do this once in your terminal)
print(f"Remember to create the custom models in your terminal using:")
print(f"ollama create {pirate_model_name} -f ./Modelfile_base")
print(f"ollama create {low_temp_model_name} -f ./Modelfile_low_temp")
print(f"ollama create {high_temp_model_name} -f ./Modelfile_high_temp")
input("Press Enter when the models are created...")

# Interaction prompts
prompts = [
    "Tell me about your ship.",
    "What is your favorite treasure?",
    "How do you navigate the seas?",
    "What do you think of other AI assistants?",
    "Translate 'hello' into pirate speak.",
    "Write a Python function to calculate the factorial of a number"
]

print(f"\n--- Interacting with {pirate_model_name} (Temperature 0.8) ---")
for prompt in prompts:
    response = run_ollama_query(pirate_model_name, prompt)
    if response:
        print(f"Prompt: {prompt}")
        print(f"Response: {response}\n")

print(f"\n--- Interacting with {low_temp_model_name} (Temperature 0.2) ---")
for prompt in prompts:
    response = run_ollama_query(low_temp_model_name, prompt)
    if response:
        print(f"Prompt: {prompt}")
        print(f"Response: {response}\n")

print(f"\n--- Interacting with {high_temp_model_name} (Temperature 1.2) ---")
for prompt in prompts:
    response = run_ollama_query(high_temp_model_name, prompt)
    if response:
        print(f"Prompt: {prompt}")
        print(f"Response: {response}\n")

print("\n--- Review the consistency of the pirate persona and the creativity of the responses across different temperatures. ---")